In [1]:
import requests
import json
import pymysql
from bs4 import BeautifulSoup as BS
import logging
import time

In [2]:
fmt = '%(asctime)s.%(msecs)03d [%(levelname)s] %(message)s'
datefmt = '%Y-%m-%d %H:%M:%S'
level = logging.INFO

In [3]:
formatter = logging.Formatter(fmt, datefmt)
logger = logging.getLogger()
logger.setLevel(level)

file = logging.FileHandler("../zhihu.log", encoding='utf-8')
file.setLevel(level)
file.setFormatter(formatter)
logger.addHandler(file)

console = logging.StreamHandler()
console.setLevel(level)
console.setFormatter(formatter)
logger.addHandler(console)

In [40]:
res = requests.get("https://www.zhihu.com/hot",headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:102.0) Gecko/20100101 Firefox/102.0","Cookie":"_zap=e40d7aab-28e0-4b4c-913f-890089f0dd7d; d_c0=\"APCfNZ0jMxWPTnqS2WvWHnRE2GLV1Il-PMs=|1656989140\"; Hm_lvt_98beee57fd2ef70ccdd5ca52b9740c49=1657110834,1657589610,1657780303,1657867448; captcha_session_v2=2|1:0|10:1657784325|18:captcha_session_v2|88:cFRRVzNMT1E5K3Z1RHdQVmR0TlBaSkRrZlJ0VXFBd0VkSGNQM2dtdDJXOEl4d0crQld4Rzl5Rkk5Qk9jQkVkZA==|9dc936750bdf67ddc8ca24dc6b3980df8782be9dd4d78eda52153435d0c6826a; __snaker__id=uGFy37d3vALoLEal; YD00517437729195%3AWM_NI=fK8WrIeSBevvj7MwDrfjWqM6Dud74NaQCrlNj7OPSRyYFf9zyOyKIXezuTg%2B4gGsxEQCsdREwCQ1YQmeR856xObMJkL1pKDhrskfZ11uVUlBl3kVXtyO%2FsAa9ydJu90nTVM%3D; YD00517437729195%3AWM_NIKE=9ca17ae2e6ffcda170e2e6ee97c85394ef8c9bf04796ac8ea7c55b968e9e86d55bf6ababd3c760aceda682e22af0fea7c3b92aa695f8b8f143a78d8e86f57eaae98398c77aa6b69992d03e88abacd8b746a38f8d93ef46aae981b1d25abb878a87bc72bb91b7accc54a7a6bbb5e621f8b98994fb4f939dbd85d14fb59d9fd2dc5bafbb9ed9b83ebc91aeaec96190efc0acc764b2ada587d93bf3ab9ab6ef4ab0eda5b7db80bb91e194c46bfba7acabf833f4b29cb7d037e2a3; YD00517437729195%3AWM_TID=Dv%2B9DP9yt5VEVQBAEUMvz7hOC78jbQ0X; gdxidpyhxdE=06W0aezaL%2BzWc9HqnacUyWy09%2BDqHb1fvGlaerUicy1T9qr9YwvEV%2BbMy8xZ7eJaBR43tm0oqL5M2nGHCiHTvDh8TzbsvTts%2B9N%2B0PD7oU47XpfP%2FY2MgeducEfN0x3Wuo2LcN1xuC0PGMycRp%2F21je6BnrD2O9HesYQhaInyG8ZmgQ8%3A1657799483632; _9755xjdesxxd_=32; captcha_ticket_v2=2|1:0|10:1657784400|17:captcha_ticket_v2|704:eyJ2YWxpZGF0ZSI6IkNOMzFfcUJ2QzRvRldydnp4STR0Mmh6bWFYMXhSNGJhaEgyYXRjLjdvX3k1Y2g1bTFnV0hnT0VwV2o4NXBnMlNsUHZGNERNbWFXU1hQbFFaNmYxZ3pKckdwc2lmazF6dnpYUldEZlZvRC5jbHF2dGE2eDF3SmQ2N2hzdmJfSWZHZDZIWUE1VEg3VlRLZ2h2SEhSbzE5bWNJcjBybzAuOWxxdTJMbl9UOHZvYUtKcEE1VW43UXpNNG1IWFpkMENYNjBEMElZZGpkRzYwV0NHclNnMmlyQ2ouNDFGTDhWU2F2ckZzWjB0Vl9iX2xhVl9HY2hLOTVENFFMQVEwMXp3cU5Cb083R1haaW9PMjZoMHpCb3JpeGs0NFNtYUtnVmhqVkVYbzFJWE13N2laY0FpanFrVXFwSnVDLURlODc3cm5aQklWb0FWTURId1o2b3lwZU1XNEt1X2VzYkkyVWJkMnhFYU0weF9MbUludEluZ3hTMmRhUy45OGlYWUlUZXJfTlpPUXBQdzJqYlpLRjJpeE91d3AwcXJTX2VjUm55bkZnLmR1N3QxSHVIOC1sbXV4dG5sbE9XWG00YVNDdVVSbHRqb0RnSjZnY2ZrSndfa2xWMUNXcjV2OHBuNjhHNUs4aVA4SGdRSG5aUC5JbG9lUGRUeWkxRGhXazBxdFJKY2djMyJ9|051c3cbb2faab6d0e119a9c14fc44c4f1aeda6b99f8c08b11936bea89184a2ff; z_c0=2|1:0|10:1657784412|4:z_c0|92:Mi4xd0lQV0JBQUFBQUFBOEo4MW5TTXpGU1lBQUFCZ0FsVk5XeGE5WXdBTXg1NV96Z2RwUzBPSklNMllWVXgzYU1uYVBR|c47d4baed05afe28ed0159c80293653eaf52ad96be4ba27958921fca5b9ebfa4; q_c1=238150b8f0c24666838561105122e78b|1657784412000|1657784412000; tst=h; _xsrf=cecc4a2d-fdff-487b-b811-6dac2205257f; KLBRSID=cdfcc1d45d024a211bb7144f66bda2cf|1657867710|1657867446; Hm_lpvt_98beee57fd2ef70ccdd5ca52b9740c49=1657867448; NOT_UNREGISTER_WAITING=1; SESSIONID=69Mx5U91LEmLpgtnBM0x2gjWwbbhIslmW0svzs03wQI; JOID=VFoSAUNwgT4SaYLXT3R74d4BLmpdQr12YFLT63sywFZbE-unJM2N4HRjg9dAHyQycwp9ItVyuRmPKRNUOWIzHEo=; osd=V18RCk5zhD0ZZIHSTH924tsCJWdeR759bVHW6HA_w1NYGOakIc6G7XdmgNxNHCExeAd-J9Z5tBqKKhhZOmcwF0c="})

In [41]:
res

<Response [200]>

In [42]:
res.text

'<!doctype html>\n<html lang="zh" data-theme="light"><head><meta charSet="utf-8"/><title data-rh="true">首页 - 知乎</title><meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=1"/><meta name="renderer" content="webkit"/><meta name="force-rendering" content="webkit"/><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"/><meta name="google-site-verification" content="FTeR0c8arOPKh8c5DYh_9uu98_zJbaWw53J-Sch9MTg"/><meta name="description" property="og:description" content="知乎，中文互联网高质量的问答社区和创作者聚集的原创内容平台，于 2011 年 1 月正式上线，以「让人们更好的分享知识、经验和见解，找到自己的解答」为品牌使命。知乎凭借认真、专业、友善的社区氛围、独特的产品机制以及结构化和易获得的优质内容，聚集了中文互联网科技、商业、影视、时尚、文化等领域最具创造力的人群，已成为综合性、全品类、在诸多领域具有关键影响力的知识分享社区和创作者聚集的原创内容平台，建立起了以社区驱动的内容变现商业模式。"/><link data-rh="true" rel="apple-touch-icon" href="https://static.zhihu.com/heifetz/assets/apple-touch-icon-152.a53ae37b.png"/><link data-rh="true" rel="apple-touch-icon" href="https://static.zhihu.com/heifetz/assets/apple-touch-icon-152.a53ae37b.png" sizes="152x152"/><link 

In [11]:
soup = BS(res.text,'lxml')

In [12]:
soup

<!DOCTYPE html>
<html data-theme="light" lang="zh"><head><meta charset="utf-8"/><title data-rh="true">首页 - 知乎</title><meta content="width=device-width,initial-scale=1,maximum-scale=1" name="viewport"/><meta content="webkit" name="renderer"/><meta content="webkit" name="force-rendering"/><meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/><meta content="FTeR0c8arOPKh8c5DYh_9uu98_zJbaWw53J-Sch9MTg" name="google-site-verification"/><meta content="知乎，中文互联网高质量的问答社区和创作者聚集的原创内容平台，于 2011 年 1 月正式上线，以「让人们更好的分享知识、经验和见解，找到自己的解答」为品牌使命。知乎凭借认真、专业、友善的社区氛围、独特的产品机制以及结构化和易获得的优质内容，聚集了中文互联网科技、商业、影视、时尚、文化等领域最具创造力的人群，已成为综合性、全品类、在诸多领域具有关键影响力的知识分享社区和创作者聚集的原创内容平台，建立起了以社区驱动的内容变现商业模式。" name="description" property="og:description"/><link data-rh="true" href="https://static.zhihu.com/heifetz/assets/apple-touch-icon-152.a53ae37b.png" rel="apple-touch-icon"/><link data-rh="true" href="https://static.zhihu.com/heifetz/assets/apple-touch-icon-152.a53ae37b.png" rel="apple-touch-icon" sizes="152x152"/><link da

In [13]:
divs= soup.find_all("section",class_ = "HotItem")
divs

[<section class="HotItem" tabindex="0"><div class="HotItem-index"><div class="HotItem-rank HotItem-hot">1</div></div><div class="HotItem-content"><a data-za-not-track-link="true" href="https://www.zhihu.com/question/543313326" rel="noopener noreferrer" target="_blank" title="银保监会回应「停贷」事件，称「引导金融机构市场化参与风险处置」，会带来哪些影响？"><h2 class="HotItem-title">银保监会回应「停贷」事件，称「引导金融机构市场化参与风险处置」，会带来哪些影响？</h2><p class="HotItem-excerpt">最近，多地业主因楼盘延期交付宣布停止还贷的事件引发广泛关注，对此，银保监会有关部门负责人今天作出回应，表示将引导金融机构市场化参与风险处置，加强与住建部门、中国人民银行工作协同，支持地方政府积极推进「保交楼、保民生、保稳定」工作。 银保监会回应「停贷」事件： 引导金融机构市场化参与风险处置 银保监会有关部门负责人表示，最近关注到个别房企楼盘出现延期交付情况，这件事的关键在于「保交楼」，对此银保监会高度重视。 多家银行发公告回应「停贷」事件： 规模较小 总体风险可控 今天，多家银行发布公告称，对近期网传「停贷」事件中涉及的楼盘展开排查，目前发布公告的银行均表示，涉及的贷款业务规模较小，不会对银行经营构成重大影响。 今天，工、农、中、建、交、邮储六家大型国有商业银行以及兴业银行、平安银行、光大银行等均发布公告，称已经对近期个别房地产企业风险暴露、住房开发项目延期交付情况展开排查，目前已发公告的多家银行均表示，涉及的业务规模较小，整体风险可控。 从银行公布的数据看，农业银行初步认定存在「保交楼」风险的楼盘涉及逾期按揭贷款余额 6.6 亿元，占全行按揭贷款余额的 0.012%； 交通银行经初步排查，存在风险的楼盘所涉及的逾期住房按揭贷款余额 0.998 亿元，占本公司境内住房按揭贷款余额 0.0067%； 邮储银行初步认定的停工项目涉及住房贷款逾期金额 1.

In [14]:
len(divs)

50

In [15]:
type(divs)

bs4.element.ResultSet

In [45]:
t2=divs[0]
t2

<section class="HotItem" tabindex="0"><div class="HotItem-index"><div class="HotItem-rank HotItem-hot">1</div></div><div class="HotItem-content"><a data-za-not-track-link="true" href="https://www.zhihu.com/question/543313326" rel="noopener noreferrer" target="_blank" title="银保监会回应「停贷」事件，称「引导金融机构市场化参与风险处置」，会带来哪些影响？"><h2 class="HotItem-title">银保监会回应「停贷」事件，称「引导金融机构市场化参与风险处置」，会带来哪些影响？</h2><p class="HotItem-excerpt">最近，多地业主因楼盘延期交付宣布停止还贷的事件引发广泛关注，对此，银保监会有关部门负责人今天作出回应，表示将引导金融机构市场化参与风险处置，加强与住建部门、中国人民银行工作协同，支持地方政府积极推进「保交楼、保民生、保稳定」工作。 银保监会回应「停贷」事件： 引导金融机构市场化参与风险处置 银保监会有关部门负责人表示，最近关注到个别房企楼盘出现延期交付情况，这件事的关键在于「保交楼」，对此银保监会高度重视。 多家银行发公告回应「停贷」事件： 规模较小 总体风险可控 今天，多家银行发布公告称，对近期网传「停贷」事件中涉及的楼盘展开排查，目前发布公告的银行均表示，涉及的贷款业务规模较小，不会对银行经营构成重大影响。 今天，工、农、中、建、交、邮储六家大型国有商业银行以及兴业银行、平安银行、光大银行等均发布公告，称已经对近期个别房地产企业风险暴露、住房开发项目延期交付情况展开排查，目前已发公告的多家银行均表示，涉及的业务规模较小，整体风险可控。 从银行公布的数据看，农业银行初步认定存在「保交楼」风险的楼盘涉及逾期按揭贷款余额 6.6 亿元，占全行按揭贷款余额的 0.012%； 交通银行经初步排查，存在风险的楼盘所涉及的逾期住房按揭贷款余额 0.998 亿元，占本公司境内住房按揭贷款余额 0.0067%； 邮储银行初步认定的停工项目涉及住房贷款逾期金额 1.2

In [81]:
ans = divs[0]
ans1=ans.find_all('a')[0]
ans1
#ans1.find('href')


<a data-za-not-track-link="true" href="https://www.zhihu.com/question/543313326" rel="noopener noreferrer" target="_blank" title="银保监会回应「停贷」事件，称「引导金融机构市场化参与风险处置」，会带来哪些影响？"><h2 class="HotItem-title">银保监会回应「停贷」事件，称「引导金融机构市场化参与风险处置」，会带来哪些影响？</h2><p class="HotItem-excerpt">最近，多地业主因楼盘延期交付宣布停止还贷的事件引发广泛关注，对此，银保监会有关部门负责人今天作出回应，表示将引导金融机构市场化参与风险处置，加强与住建部门、中国人民银行工作协同，支持地方政府积极推进「保交楼、保民生、保稳定」工作。 银保监会回应「停贷」事件： 引导金融机构市场化参与风险处置 银保监会有关部门负责人表示，最近关注到个别房企楼盘出现延期交付情况，这件事的关键在于「保交楼」，对此银保监会高度重视。 多家银行发公告回应「停贷」事件： 规模较小 总体风险可控 今天，多家银行发布公告称，对近期网传「停贷」事件中涉及的楼盘展开排查，目前发布公告的银行均表示，涉及的贷款业务规模较小，不会对银行经营构成重大影响。 今天，工、农、中、建、交、邮储六家大型国有商业银行以及兴业银行、平安银行、光大银行等均发布公告，称已经对近期个别房地产企业风险暴露、住房开发项目延期交付情况展开排查，目前已发公告的多家银行均表示，涉及的业务规模较小，整体风险可控。 从银行公布的数据看，农业银行初步认定存在「保交楼」风险的楼盘涉及逾期按揭贷款余额 6.6 亿元，占全行按揭贷款余额的 0.012%； 交通银行经初步排查，存在风险的楼盘所涉及的逾期住房按揭贷款余额 0.998 亿元，占本公司境内住房按揭贷款余额 0.0067%； 邮储银行初步认定的停工项目涉及住房贷款逾期金额 1.27 亿元； 兴业银行涉及「停贷」事件楼盘相关按揭贷款余额 16 亿元（其中不良贷款余额 0.75 亿元），已出现停止还款的按揭金额 3.84 亿元。最新！银保监会回应「停贷」事件 _ 央广网</p></a>

In [82]:
q={}
q['href']=ans1['href']
q['title']=ans1['title']
q

{'href': 'https://www.zhihu.com/question/543313326',
 'title': '银保监会回应「停贷」事件，称「引导金融机构市场化参与风险处置」，会带来哪些影响？'}

In [83]:
ans1.find('p')

<p class="HotItem-excerpt">最近，多地业主因楼盘延期交付宣布停止还贷的事件引发广泛关注，对此，银保监会有关部门负责人今天作出回应，表示将引导金融机构市场化参与风险处置，加强与住建部门、中国人民银行工作协同，支持地方政府积极推进「保交楼、保民生、保稳定」工作。 银保监会回应「停贷」事件： 引导金融机构市场化参与风险处置 银保监会有关部门负责人表示，最近关注到个别房企楼盘出现延期交付情况，这件事的关键在于「保交楼」，对此银保监会高度重视。 多家银行发公告回应「停贷」事件： 规模较小 总体风险可控 今天，多家银行发布公告称，对近期网传「停贷」事件中涉及的楼盘展开排查，目前发布公告的银行均表示，涉及的贷款业务规模较小，不会对银行经营构成重大影响。 今天，工、农、中、建、交、邮储六家大型国有商业银行以及兴业银行、平安银行、光大银行等均发布公告，称已经对近期个别房地产企业风险暴露、住房开发项目延期交付情况展开排查，目前已发公告的多家银行均表示，涉及的业务规模较小，整体风险可控。 从银行公布的数据看，农业银行初步认定存在「保交楼」风险的楼盘涉及逾期按揭贷款余额 6.6 亿元，占全行按揭贷款余额的 0.012%； 交通银行经初步排查，存在风险的楼盘所涉及的逾期住房按揭贷款余额 0.998 亿元，占本公司境内住房按揭贷款余额 0.0067%； 邮储银行初步认定的停工项目涉及住房贷款逾期金额 1.27 亿元； 兴业银行涉及「停贷」事件楼盘相关按揭贷款余额 16 亿元（其中不良贷款余额 0.75 亿元），已出现停止还款的按揭金额 3.84 亿元。最新！银保监会回应「停贷」事件 _ 央广网</p>

In [85]:
ans1.find('p').contents[0]

'最近，多地业主因楼盘延期交付宣布停止还贷的事件引发广泛关注，对此，银保监会有关部门负责人今天作出回应，表示将引导金融机构市场化参与风险处置，加强与住建部门、中国人民银行工作协同，支持地方政府积极推进「保交楼、保民生、保稳定」工作。 银保监会回应「停贷」事件： 引导金融机构市场化参与风险处置 银保监会有关部门负责人表示，最近关注到个别房企楼盘出现延期交付情况，这件事的关键在于「保交楼」，对此银保监会高度重视。 多家银行发公告回应「停贷」事件： 规模较小 总体风险可控 今天，多家银行发布公告称，对近期网传「停贷」事件中涉及的楼盘展开排查，目前发布公告的银行均表示，涉及的贷款业务规模较小，不会对银行经营构成重大影响。 今天，工、农、中、建、交、邮储六家大型国有商业银行以及兴业银行、平安银行、光大银行等均发布公告，称已经对近期个别房地产企业风险暴露、住房开发项目延期交付情况展开排查，目前已发公告的多家银行均表示，涉及的业务规模较小，整体风险可控。 从银行公布的数据看，农业银行初步认定存在「保交楼」风险的楼盘涉及逾期按揭贷款余额 6.6 亿元，占全行按揭贷款余额的 0.012%； 交通银行经初步排查，存在风险的楼盘所涉及的逾期住房按揭贷款余额 0.998 亿元，占本公司境内住房按揭贷款余额 0.0067%； 邮储银行初步认定的停工项目涉及住房贷款逾期金额 1.27 亿元； 兴业银行涉及「停贷」事件楼盘相关按揭贷款余额 16 亿元（其中不良贷款余额 0.75 亿元），已出现停止还款的按揭金额 3.84 亿元。最新！银保监会回应「停贷」事件 _ 央广网'